In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

## Weather Data Ingestion into Hopsworks


In [2]:
# read the data
csv_file=Path("./guangzhou-air-quality.csv")
if csv_file.is_file():
    print("File exists")
else:
    print("Not exists")

File exists


In [3]:
country="china"
city = "guangzhou"
street = "shiwuzhong"
aqicn_url="https://api.waqi.info/feed/@9846"

# This API call may fail if the IP address you run this notebook from is blocked by the Nominatim API
# If this fails, lookup the latitude, longitude using Google and enter the values here.
latitude, longitude = 23.1291, 113.2644

today = datetime.date.today()

In [4]:
aqi_api_key_file = Path("./aqi-api-key.txt")
if csv_file.is_file():
    print("File exists")
else:
    print("Not exists")

with open(aqi_api_key_file, 'r') as file:
    AQI_API_KEY = file.read().rstrip()

File exists


In [5]:
with open('./hopsworks-api-key.txt', 'r') as file:
     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157273


In [6]:
host = "c.app.hopsworks.ai"
api_key = os.environ.get('HOPSWORKS_API_KEY')
conn = hopsworks.connection(host=host, project=project.name, api_key_value=api_key)
secrets=conn.get_secrets_api()
try:
    secrets.create_secret("AQI_API_KEY", AQI_API_KEY)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

Connected. Call `.close()` to terminate connection gracefully.


In [7]:
url = f"{aqicn_url}/?token={AQI_API_KEY}"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve data. Status Code:", response.status_code)
    raise requests.exceptions.RequestException(response.status_code)

In [8]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df = df.rename(columns={"median": "pm25"})
df = df.rename(columns={"timestamp": "date"})

df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')

df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916 entries, 0 to 3915
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3916 non-null   datetime64[ns]
 1   pm25    3903 non-null   float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 46.0 KB


In [9]:
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url
df_aq

,date,pm25,country,city,street,url
0,2024-11-01,88.0,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
1,2024-11-02,77.0,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
2,2024-11-03,99.0,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
3,2024-11-04,100.0,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
4,2024-11-05,77.0,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
...,...,...,...,...,...,...
3911,2017-01-01,NaN,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
3912,2016-01-03,NaN,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
3913,2015-09-22,NaN,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846
3914,2014-12-31,NaN,china,guangzhou,shiwuzhong,https://api.waqi.info/feed/@9846


In [10]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

'2013-12-31'

In [11]:
import requests_cache
from retry_requests import retry
import openmeteo_requests

def get_historical_weather(city, start_date,  end_date, latitude, longitude):
    # latitude, longitude = get_city_coordinates(city)

    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": ["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max", "wind_direction_10m_dominant"]
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(2).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(3).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s"),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

    daily_dataframe = pd.DataFrame(data = daily_data)
    daily_dataframe = daily_dataframe.dropna()
    daily_dataframe['city'] = city
    return daily_dataframe

In [12]:
weather_df = get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

Coordinates 23.163442611694336°N 113.2774887084961°E
Elevation 17.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [13]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3968 entries, 0 to 3967
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         3968 non-null   datetime64[ns]
 1   temperature_2m_mean          3968 non-null   float32       
 2   precipitation_sum            3968 non-null   float32       
 3   wind_speed_10m_max           3968 non-null   float32       
 4   wind_direction_10m_dominant  3968 non-null   float32       
 5   city                         3968 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 155.0+ KB


In [14]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

In [15]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

In [16]:
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


In [17]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.
{"country": "china", "city": "guangzhou", "street": "shiwuzhong", "aqicn_url": "https://api.waqi.info/feed/@9846", "latitude": 23.1291, "longitude": 113.2644}


In [18]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

In [19]:
air_quality_fg.insert(df_aq)

2024-11-12 10:48:42,095 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157273/fs/1147976/fg/1337686


Uploading Dataframe: 0.00% |          | Rows 0/3916 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157273/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc0d4075880>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 661508
         }
       },
       "result": {
         "observed_value": 9.0,
         "element_count": 3916,
         "missing_count": 13,
         "missing_percent": 0.3319713993871297
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T02:48:42.000095Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations"

In [20]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [23]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city','date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

In [24]:
weather_fg.insert(weather_df)

2024-11-12 10:50:34,531 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157273/fs/1147976/fg/1338705


Uploading Dataframe: 0.00% |          | Rows 0/3968 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157273/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc07c1e2dc0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 662529
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 3968,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T02:50:34.000530Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_be

In [25]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")

In [ ]:
df['pm25_rolling_avg'] = df['pm25'].rolling(3).mean()